In [1]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Initialize an empty DataFrame
all_data = pd.DataFrame()


In [ ]:
while True:
    # Prompt for the file path
    file_path = input("./annotation/n_dialog_11_0.txt/INITIAL_CAS/INITIAL_CAS.xmi")
    
    if file_path.lower() == 'done':
        break
    
    if not os.path.isfile(file_path):
        print(f"File {file_path} does not exist. Please try again.")
        continue
    
    try:
        # Parse the XML file
        with open(file_path, 'r', encoding='utf-8') as file:
            xml_content = file.read()
            root = ET.fromstring(xml_content)
            
            # Extract the sofaString
            sofa_string = root.attrib.get('sofaString', '')
            
            # Split the sofaString by line breaks (&#10;)
            lines = sofa_string.split('&#10;&#10;')
            
            if len(lines) >= 5:  # Ensure there are enough lines to extract the required information
                # Extract the directory name
                directory_name = os.path.dirname(file_path)
                

                # Extract the required information
                data = {
                    'Directory': directory_name,
                    'Nationality': lines[0],
                    'Gender': lines[3],
                    'Training Center': lines[4]
                }
                
                # Append the data to the DataFrame
                all_data = all_data.append(data, ignore_index=True)
            else:
                print(f"File {file_path} does not contain enough data.")
    
    except Exception as e:
        print(f"An error occurred while processing file {file_path}: {e}")

In [ ]:
# Save the DataFrame to an Excel file if there is any data
if not all_data.empty:
    output_file = 'all_data_output.xlsx'
    all_data.to_excel(output_file, index=False)
    print(f'Data saved to {output_file}')
else:
    print("No data to save.")
